# News-Link: Proccess Executed in App Server File

This notebook highlights the thought process behind how **News-link** is run using a server file. **News-Link** (and by extension) this document are run using several functions, written and stored in another function file.
Note this notebooke was created prior to the data cleaning process was established - hence the loading and cleaning of data here (one can load in the full news corpus instead). 

In [24]:
#Importing required packages
import pandas as pd
import numpy as np
import sys
from pathlib import Path

#specifying the root path
sys.path.insert(1, '../scripts/')

#self written functions and scripts that will be needed
import text_normalization_funs as TN
import similiarity_funcs as SIM

In [25]:
#Specifying root path and path to data
root_path = Path('..')
data_path = root_path/"data"
scripts_path = data_path/"scripts"

In [7]:
#Loading and cleaning data files scraped over 3 different days, dropping NAs at load
newsJan20 = pd.read_csv(data_path/"raw"/'Jan_20_news.csv').dropna()
newsJan21 = pd.read_csv(data_path/"raw"/'Jan_21_news.csv').dropna()
newsJan23 = pd.read_csv(data_path/"raw"/'Jan_23_news.csv').dropna()

#Concatinating all dataframes
news = [newsJan20, newsJan21, newsJan23]
combine_news = pd.concat(news)

In [8]:
#Likely some duplicate stories references over different days and different landing pages
#Removing said duplicates
unique_news = combine_news.drop_duplicates(subset = 'mainurl')

## Text Pre-Processing

Following analyses are for preprocessing text to combine and normalize it for TF-IDF. As noted above, this document was created prior to the full cleaned corpus for a mid-project demo. As such, this section can be skipped and the full corpus could just be loaded in. 

In [9]:
#Combing the news story headlines with the maintext for the stories
full_text = unique_news["title"].map(str)+ '. ' + unique_news["maintext"]

#Preprocessing all text to conduct tl-dlf cosine similarity analysis
normfull = TN.normalize_NewsText(full_text)

In [10]:
#Adding in cleaned text to dataframe
unique_news['full_text'] = full_text
unique_news['cleaned_text'] = normfull

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
#Isolating the first 30% of a news story, based on cleaned full text
reduced_text = []

for doc in unique_news['cleaned_text']:
    redu = TN.Clean30(doc)
    reduced_text.append(redu)

unique_news["reduced"] = reduced_text

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
#Exporting the cleaned dataframe as its own
unique_news.to_csv(data_path/"processed"/'demo_corp.csv')

## Computing Similarity Scores

After creation of demo corp, splitting off a test story and testing similarity functions

In [17]:
#From the cleaned text - creating a corpus and a test story
corpus = unique_news.iloc[1:len(unique_news)]
story = unique_news.iloc[0]

In [29]:
#Computing similarity scores for given dataframe
scores = SIM.SimCorpCreate (corpora = corpus['cleaned_text'], directory= str(scripts_path), querydoc = story['cleaned_text'])

In [31]:
#Getting list of 10 most similar stories in the corpus
SimList = SIM.StoryMatch(SimScores = scores, originalDF = corpus)

../scripts/similiarity_funcs.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  originalDF['sim_scores'] = SimScores


In [32]:
#Displaying the result of the similarity functions (i.e. most similar news stories identified in corpus by the model)
SimList.head()

,title,mainurl,sim_scores
422,Snowfall warning in effect for P.E.I. on Sunday,https://www.cbc.ca/news/canada/prince-edward-i...,0.343621
361,Flurries expected for Monday morning on P.E.I.,https://www.cbc.ca/news/canada/prince-edward-i...,0.339433
109,"With snowy, windy conditions, RCMP advise Isla...",https://www.cbc.ca/news/canada/prince-edward-i...,0.324324
333,Poor road conditions cause school closures in ...,https://www.cbc.ca/news/canada/new-brunswick/w...,0.211661
313,Week should warm after bitter beginning,https://www.cbc.ca/news/canada/ottawa/ottawa-g...,0.206741
